# 线性回归的简洁实现

通过使用深度学习框架来简洁地实现
线性回归模型
生成数据集

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

调用框架中现有的API来读取数据

In [5]:
def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器。"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 0.1726, -0.4128],
         [-0.7083, -0.5516],
         [-0.3504,  0.2673],
         [-1.4505,  0.6314],
         [-0.0310, -1.8659],
         [ 1.2988,  0.5918],
         [-1.7788, -0.1763],
         [ 0.1456,  1.4922],
         [ 0.2423,  0.9503],
         [ 1.5332, -0.0598]]),
 tensor([[ 5.9472],
         [ 4.6643],
         [ 2.5635],
         [-0.8511],
         [10.4815],
         [ 4.7820],
         [ 1.2355],
         [-0.5880],
         [ 1.4607],
         [ 7.4764]])]

使用框架的预定义好的层

In [6]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

初始化模型参数

In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

计算均方误差使用的是`MSELoss`类，也称为平方 $L_2$ 范数

In [8]:
loss = nn.MSELoss()

实例化 `SGD` 实例

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

训练过程代码与我们从零开始实现时所做的非常相似

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000264
epoch 2, loss 0.000096
epoch 3, loss 0.000096


比较生成数据集的真实参数和通过有限数据训练获得的模型参数

In [11]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0004,  0.0006])
b的估计误差： tensor([0.0004])
